# 📱 Phone Status Monitor Simulator  

This project is a **Python OOP simulation** of a mobile phone that keeps track of its **battery, balance, and data**.  
It allows you to perform everyday phone actions and see how the phone’s state changes in real time.  

---

## 🔑 Key Features  
- **Battery Monitoring** → charge the phone (by time or percent).  
- **Balance Tracking** → top up in dollars and use it for calls or data.  
- **Data Management** → buy data bundles with balance.  
- **Call Simulation** → make calls that reduce balance and battery.  
- **Browsing Simulation** → browse the internet based on duration and profile:  
  - *Light* (chatting, light apps)  
  - *Normal* (social media)  
  - *Heavy* (video streaming)  
- **Status Display** → check the phone’s current state with a simple battery bar.  

---

## 🎯 Learning Goals  
This project helps you understand **Object-Oriented Programming (OOP)** in Python by showing how:  
- **Classes** act as blueprints.  
- **Objects** represent real things (your phone).  
- **Attributes** store state (battery, balance, data).  
- **Methods** define actions (charge, call, browse).  


In [1]:
# --- Phone Status Monitor Simulator (improved OOP) ---

class Phone:
    # ===== Configurable "tariffs" / rates =====
    CURRENCY = "$"
    DATA_MB_PER_DOLLAR = 2          # $1 -> 2 MB
    CALL_COST_PER_MIN = 1.0         # $ per minute
    CALL_BATTERY_PER_MIN = 1        # % battery per minute on call 2

    # Browsing profiles: data+ battery usage per minute
    BROWSE_PROFILES = {
        "light":  {"mb_per_min": 2,  "battery_per_min": 0.5},  # chats, light apps 
        "normal": {"mb_per_min": 5,  "battery_per_min": 1},  # social media 
        "heavy":  {"mb_per_min": 20, "battery_per_min": 2},  # streaming/video 
    }

    CHARGE_RATE_PER_MIN = 1.0       # % battery per minute on charger

    def __init__(self, owner, battery=50, balance=0.0, data_mb=0):
        self.owner = owner
        self.battery = int(battery)    # 0–100
        self.balance = float(balance)  # dollars
        self.data_mb = int(data_mb)    # megabytes

    # ===== Helpers =====
    def _cap_battery(self):
        self.battery = max(0, min(100, int(self.battery)))

    def _require(self, cond, msg):
        if not cond:
            print(f"⚠️  {msg}")
            return False
        return True

    def _bar(self, value, max_value=100, width=20):
        # simple text progress bar for nicer status
        filled = int(round((value / max_value) * width))
        return "[" + "█" * filled + "·" * (width - filled) + "]"

    # ===== Actions =====
    def charge(self, minutes=None, percent=None):
        """
        Charge by time (preferred for realism) or by raw percent.
        """
        if minutes is not None:
            if not self._require(minutes > 0, "Charge minutes must be > 0."):
                return
            self.battery += minutes * self.CHARGE_RATE_PER_MIN
        elif percent is not None:
            if not self._require(percent > 0, "Charge percent must be > 0."):
                return
            self.battery += percent
        else:
            print("⚠️  Provide either minutes= or percent= to charge().")
            return
        self._cap_battery()
        print(f"🔌 Charging... Battery: {self.battery}% {self._bar(self.battery)}")

    def top_up(self, amount):
        if not self._require(amount > 0, "Top-up amount must be > 0."):
            return
        self.balance += amount
        print(f"💳 Topped up {self.CURRENCY}{amount:.2f}. Balance: {self.CURRENCY}{self.balance:.2f}")

    def buy_data(self, dollars):
        if not self._require(dollars > 0, "Purchase must be > 0."):
            return
        if not self._require(dollars <= self.balance, "Not enough balance to buy data."):
            return
        self.balance -= dollars
        gained = int(dollars * self.DATA_MB_PER_DOLLAR)
        self.data_mb += gained
        print(f"📶 Bought {gained}MB. Data: {self.data_mb}MB | Balance: {self.CURRENCY}{self.balance:.2f}")

    def call(self, minutes):
        if not self._require(minutes > 0, "Call minutes must be > 0."):
            return
        cost = minutes * self.CALL_COST_PER_MIN
        battery_use = minutes * self.CALL_BATTERY_PER_MIN

        if not self._require(cost <= self.balance, "Call failed: not enough balance."):
            return
        if not self._require(battery_use <= self.battery, "Call failed: low battery."):
            return

        self.balance -= cost
        self.battery -= battery_use
        self._cap_battery()
        print(f"📞 Called {minutes} min. "
              f"Balance: {self.CURRENCY}{self.balance:.2f} | Battery: {self.battery}% {self._bar(self.battery)}")

    def browse(self, minutes, profile="normal"):
        """
        Duration-based browsing.
        - minutes: how long online
        - profile: 'light' | 'normal' | 'heavy' (controls MB + battery per minute)
        """
        if not self._require(minutes > 0, "Browse minutes must be > 0."):
            return
        if profile not in self.BROWSE_PROFILES:
            print(f"⚠️  Unknown profile '{profile}'. Use one of: {list(self.BROWSE_PROFILES)}")
            return

        mb_per_min = self.BROWSE_PROFILES[profile]["mb_per_min"]
        batt_per_min = self.BROWSE_PROFILES[profile]["battery_per_min"]

        need_mb = minutes * mb_per_min
        need_batt = minutes * batt_per_min

        if not self._require(need_mb <= self.data_mb, "Browse failed: not enough data."):
            return
        if not self._require(need_batt <= self.battery, "Browse failed: low battery."):
            return

        self.data_mb -= need_mb
        self.battery -= need_batt
        self._cap_battery()
        print(f"🌐 Browsed {minutes} min ({profile}). "
              f"Used {need_mb}MB, {need_batt}% battery. "
              f"Data: {self.data_mb}MB | Battery: {self.battery}% {self._bar(self.battery)}")

    def status(self):
        print(
            f"📱 {self.owner}'s Phone\n"
            f"   Battery: {self.battery}% {self._bar(self.battery)}\n"
            f"   Balance: {self.CURRENCY}{self.balance:.2f}\n"
            f"   Data:    {self.data_mb}MB"
        )



## 📱 Functionalities of the `Phone` Class

The `Phone` class simulates a mobile phone with the following functionalities:

- **Status**
  - Display the current phone state: battery %, balance ($), and data (MB).
  - Includes a simple battery progress bar.

- **Charge**
  - Charge the phone by **minutes** (realistic charging).
  - Optionally charge by **percentage**.
  - Battery is capped between 0% and 100%.

- **Top Up**
  - Add balance in **dollars ($)**.

- **Buy Data**
  - Use balance to purchase data bundles.
  - Conversion rate: **$1 → 2 MB** (default).

- **Call**
  - Make calls for a given number of minutes.
  - Reduces **balance** ($1 per minute).
  - Consumes **battery** (2% per minute).
  - Fails if balance or battery is insufficient.

- **Browse**
  - Browse online for a given duration (minutes).
  - Choose a **profile**:  
    - *Light* → 2 MB/min, 1% battery/min  
    - *Normal* → 5 MB/min, 2% battery/min  
    - *Heavy* → 20 MB/min, 4% battery/min  
  - Reduces **data** and **battery** accordingly.
  - Fails if data or battery is insufficient.


## 📱 Functionalities of the `Phone` Class and How to Call Them

### 1. Check Status
See the phone’s current battery, balance, and data.

```python
my_phone.status()
```

### 2. Charge the Phone

Increase the battery (capped at 100%).

- By minutes (realistic charging):

```python
my_phone.charge(minutes=20)
```

```python
my_phone.charge(percent=30)
```

### 3. Top Up Balance

Add money (in dollars) to your balance.
```python
my_phone.top_up(10)   # adds $10
```


### 4. Buy Data

Use balance to buy data bundles.

Rate: $1 → 2 MB
```python
my_phone.buy_data(5)   # uses $5 → adds 10MB
```

### 5. Make a Call

Spend balance and battery based on call duration.

- Cost: $1 per minute

- Battery: 2% per minute

```python
my_phone.call(3)   # 3-minute call → costs $3, uses 6% battery
```

### 6. Browse the Internet

Use data and battery for a set duration (minutes).

Choose a profile: "light", "normal", "heavy"

- Light → 2 MB/min, 1% battery/min

- Normal → 5 MB/min, 2% battery/min

- Heavy → 20 MB/min, 4% battery/min

```python
my_phone.browse(10, profile="normal")   # 10 min normal browsing
my_phone.browse(5, profile="heavy")     # 5 min heavy browsing
```

In [2]:
my_phone = Phone("Zibah", battery=45, balance=780, data_mb=3500)

In [3]:
my_phone.status()

📱 Zibah's Phone
   Battery: 45% [█████████···········]
   Balance: $780.00
   Data:    3500MB


In [4]:
my_phone.buy_data(15)

📶 Bought 30MB. Data: 3530MB | Balance: $765.00


In [5]:
my_phone.status()

📱 Zibah's Phone
   Battery: 45% [█████████···········]
   Balance: $765.00
   Data:    3530MB


In [6]:
my_phone.call(4)

📞 Called 4 min. Balance: $761.00 | Battery: 41% [████████············]


In [7]:
my_phone.status()

📱 Zibah's Phone
   Battery: 41% [████████············]
   Balance: $761.00
   Data:    3530MB


In [8]:
my_phone.browse(6, profile="heavy") 

🌐 Browsed 6 min (heavy). Used 120MB, 12% battery. Data: 3410MB | Battery: 29% [██████··············]


In [9]:
my_phone.charge(minutes=20)

🔌 Charging... Battery: 49% [██████████··········]


In [10]:
my_phone.charge(percent=30)

🔌 Charging... Battery: 79% [████████████████····]


In [11]:
my_phone.status()

📱 Zibah's Phone
   Battery: 79% [████████████████····]
   Balance: $761.00
   Data:    3410MB


In [12]:
my_phone.charge(minutes=5, percent=10)

🔌 Charging... Battery: 84% [█████████████████···]


In [13]:
my_phone.status()

📱 Zibah's Phone
   Battery: 84% [█████████████████···]
   Balance: $761.00
   Data:    3410MB
